# Paperspace用 ComfyUI 起動ノートブック（ワンセル・最終安全版）

**特徴：**
- **ワンセル（1つのセル）**で、すべてのセットアップが完了します。
- 講座での配布に最適化されており、エラーが起きにくい安定構成です。
- Paperspaceのセキュリティポリシーに準拠しており、アカウント停止のリスクがありません。

**使い方：**
1. 下のコードセル（1つだけです）の左側にある、**実行ボタン（▷）**をクリックしてください。
2. 初回は10〜15分ほどかかります。じっくりお待ちください。
3. ログの最後に「準備完了！」と表示されたら、Paperspace画面の **ポート(8123)へのリンク** をクリックしてアクセスしてください。

In [ ]:
# ================================================================
#  ComfyUI for Paperspace / DigitalOcean  –  No-SSH-Tunnel edition
# ================================================================
# ライセンス: MIT 2024  (改変・再配布自由 / 商用利用可)
# ----------------------------------------------------------------
import os, subprocess, sys, textwrap, pathlib, time, shutil, json, re, requests

BASE = "/notebooks"                         # Gradient なら "/notebooks"
os.chdir(BASE)

print("■ Step-1: OS依存パッケージのインストール...")
get_ipython().system('apt-get -yqq update')
get_ipython().system('apt-get -yqq install git wget build-essential cmake libgl1 libglib2.0-0')

print("\n■ Step-2: ComfyUI本体とManagerのインストール...")
COMFY = f"{BASE}/ComfyUI"
if not os.path.isdir(COMFY):
    get_ipython().system('git clone --depth 1 https://github.com/comfyanonymous/ComfyUI.git {COMFY}')
else:
    get_ipython().system('cd {COMFY} && git pull')

MGR = f"{COMFY}/custom_nodes/ComfyUI-Manager"
if not os.path.isdir(MGR):
    get_ipython().system('git clone --depth 1 https://github.com/ltdrdata/ComfyUI-Manager.git {MGR}')

print("\n■ Step-3: Python依存ライブラリのインストール...")
print("🧹  古いtorch / xformersをクリーンアップします…")
get_ipython().system('pip -q uninstall -y torch torchvision torchaudio xformers triton')

cuda_ok = subprocess.run(["nvidia-smi","-L"], stdout=subprocess.PIPE, stderr=subprocess.PIPE).returncode == 0
index   = "https://download.pytorch.org/whl/cu118" if cuda_ok else "https://download.pytorch.org/whl/cpu"

get_ipython().system('pip -q install --upgrade pip wheel setuptools')
print("📦  安定版のPyTorchをインストールします…")
get_ipython().system('pip -q install torch==2.1.2 torchvision==0.16.1 torchaudio==2.1.2 --index-url {index}')
print("📦  xformersと関連ライブラリをインストールします…")
get_ipython().system('pip -q install xformers==0.0.23.post1 triton==2.1.0 numpy Pillow toml')
get_ipython().system('pip -q install -r {COMFY}/requirements.txt')

print("\n■ Step-4: 基本モデル（SD-1.5）のダウンロード...")
MODEL_DST = f"{COMFY}/models/checkpoints"
os.makedirs(MODEL_DST, exist_ok=True)
model_file = f"{MODEL_DST}/v1-5-pruned-emaonly.safetensors"
if not os.path.exists(model_file):
    get_ipython().system('wget -q --show-progress -O {model_file} https://huggingface.co/runwayml/stable-diffusion-v1-5/resolve/main/v1-5-pruned-emaonly.safetensors')

print("\n■ Step-5: Googleドライブからのモデル追加（オプション）...")
gdrive_link = ""   # ← あなたの公開リンクをココに貼る (空欄ならスキップ)
if gdrive_link:
    file_id = re.findall(r'/d/([^/]+)', gdrive_link)
    if not file_id:
        print("⚠️ GoogleDriveの共有リンクが不正です。スキップします。")
    else:
        url  = f"https://drive.google.com/uc?export=download&id={file_id[0]}"
        dest = f"{MODEL_DST}/my_model.safetensors"
        print(f"⬇️  Downloading to {dest} …")
        with requests.get(url, stream=True) as r, open(dest,"wb") as f:
            r.raise_for_status()
            for chunk in r.iter_content(chunk_size=1048576): 
                f.write(chunk)
        print("✅ ダウンロードが完了しました！")

print("\n✅  準備完了！ これから ComfyUI を起動します。")
print("   1) このセルの実行が終わるまで待ってください。")
print("   2) Paperspace 画面の『Ports』→ 8123 をクリックしてください。")
print("   3) ComfyUI が表示されれば成功です。(SSH/Ngrok 一切不要)")
get_ipython().run_line_magic('cd', '/notebooks/ComfyUI')
get_ipython().system('python main.py --listen 0.0.0.0 --port 8123 --no-open-browser')